In [0]:
secret=dbutils.secrets.get('databricklogin','databricklogin')

In [0]:
application_id ="88dec17f-62f0-4cab-82b7-b9022c2cc21c"
service_credential=secret
directory_id="037e0179-cd0e-4201-951c-167b7554d77a"
spark.conf.set("fs.azure.account.auth.type.datastoragenew12.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datastoragenew12.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datastoragenew12.dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.datastoragenew12.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datastoragenew12.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
display(dbutils.fs.ls("abfss://raw@datastoragenew12.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://raw@datastoragenew12.dfs.core.windows.net/Address/,Address/,0,1732920488000
abfss://raw@datastoragenew12.dfs.core.windows.net/Customer/,Customer/,0,1732920489000
abfss://raw@datastoragenew12.dfs.core.windows.net/CustomerAddress/,CustomerAddress/,0,1732920492000
abfss://raw@datastoragenew12.dfs.core.windows.net/Product/,Product/,0,1732920478000
abfss://raw@datastoragenew12.dfs.core.windows.net/ProductCategory/,ProductCategory/,0,1732920495000
abfss://raw@datastoragenew12.dfs.core.windows.net/ProductDescription/,ProductDescription/,0,1732920492000
abfss://raw@datastoragenew12.dfs.core.windows.net/ProductModel/,ProductModel/,0,1732920478000
abfss://raw@datastoragenew12.dfs.core.windows.net/ProductModelProductDescription/,ProductModelProductDescription/,0,1732920494000
abfss://raw@datastoragenew12.dfs.core.windows.net/SalesOrderDetail/,SalesOrderDetail/,0,1732920489000
abfss://raw@datastoragenew12.dfs.core.windows.net/SalesOrderHeader/,SalesOrderHeader/,0,1732920493000


In [0]:
df_a = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@datastoragenew12.dfs.core.windows.net/Address/Address.csv")
display(df_a)

In [0]:

from pyspark.sql.functions import col, date_format

# Extract the date from the ModifiedDate column and create a new column 'Date' and drop the original column 'ModifiedDate'
df_ac = df_a.withColumn("Date", date_format(col("ModifiedDate"), "yyyy-MM-dd")) \
                                                                            .drop("ModifiedDate")
         

display(df_ac)

In [0]:
df_ac.write.format("delta").mode("append").save("abfss://proccesed@datastoragenew12.dfs.core.windows.net/Address")

In [0]:
df_c = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@datastoragenew12.dfs.core.windows.net/Customer/Customer.csv")
display(df_c)

In [0]:
df_cc = df_c.withColumn("Date", date_format(col("ModifiedDate"), "yyyy-MM-dd")) \
                                                                            .drop("ModifiedDate")
display(df_cc)   
df_cc.write.format("delta").mode("append").save("abfss://proccesed@datastoragenew12.dfs.core.windows.net/Customer")

In [0]:
df_ca = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@datastoragenew12.dfs.core.windows.net/CustomerAddress/CustomerAddress.csv")
display(df_ca)

In [0]:
df_cac = df_ca.withColumn("Date", date_format(col("ModifiedDate"), "yyyy-MM-dd")) \
                                                                            .drop("ModifiedDate")
display(df_cac)   
df_cac.write.format("delta").mode("append").save("abfss://proccesed@datastoragenew12.dfs.core.windows.net/CustomerAddress")

In [0]:

df_p = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("abfss://raw@datastoragenew12.dfs.core.windows.net/Product/Product.csv")
display(df_p)

In [0]:
df_pc = df_p.withColumn("Date", date_format(col("ModifiedDate"), "yyyy-MM-dd")) \
                                                                            .drop("ModifiedDate")
df_pc = df_pc.withColumn("SellStartDate", date_format(col("SellStartDate"), "yyyy-MM-dd")) \
            .withColumn("SellEndDate", date_format(col("SellEndDate"), "yyyy-MM-dd"))                                                                          
display(df_pc)   
df_pc.write.format("delta") \
            .mode("append") \
            .save("abfss://proccesed@datastoragenew12.dfs.core.windows.net/Product")